## Enron Data POI Classifier 
### Jo Anna Capp

In the 1990's Enron Corporation was one of the largest.....


In [1]:
#set working directory
import os
os.chdir('D:/Documents/Udacity/IntroMachineLearning/ud120projectsmaster/ud120projectsmaster/UdacityP5')

In [2]:
#import all packages and modules here
import sys
import pickle
sys.path.append("../tools/")
import pandas
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from ggplot import *

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

In [3]:
features_list = ['poi']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

Lets look at the structure of the dataset and check for missing values.

In [4]:
#total individuals
print "There are ", len(data_dict.keys()), "executives of interest in the Enron dataset"
#number of pois
num_poi = 0
for dic in data_dict.values():
    if dic['poi'] == 1: 
        num_poi += 1
print "There are ", num_poi, "identified persons of interest within the dataset"
print "Data Dictionary Keys:"
print(data_dict.keys())
#data dictionary format
print "A typical key:value list: ", data_dict["SKILLING JEFFREY K"]


There are  146 executives of interest in the Enron dataset
There are  18 identified persons of interest within the dataset
Data Dictionary Keys:
['METTS MARK', 'BAXTER JOHN C', 'ELLIOTT STEVEN', 'CORDES WILLIAM R', 'HANNON KEVIN P', 'MORDAUNT KRISTINA M', 'MEYER ROCKFORD G', 'MCMAHON JEFFREY', 'HORTON STANLEY C', 'PIPER GREGORY F', 'HUMPHREY GENE E', 'UMANOFF ADAM S', 'BLACHMAN JEREMY M', 'SUNDE MARTIN', 'GIBBS DANA R', 'LOWRY CHARLES P', 'COLWELL WESLEY', 'MULLER MARK S', 'JACKSON CHARLENE R', 'WESTFAHL RICHARD K', 'WALTERS GARETH W', 'WALLS JR ROBERT H', 'KITCHEN LOUISE', 'CHAN RONNIE', 'BELFER ROBERT', 'SHANKMAN JEFFREY A', 'WODRASKA JOHN', 'BERGSIEKER RICHARD P', 'URQUHART JOHN A', 'BIBI PHILIPPE A', 'RIEKER PAULA H', 'WHALEY DAVID A', 'BECK SALLY W', 'HAUG DAVID L', 'ECHOLS JOHN B', 'MENDELSOHN JOHN', 'HICKERSON GARY J', 'CLINE KENNETH W', 'LEWIS RICHARD', 'HAYES ROBERT E', 'MCCARTY DANNY J', 'KOPPER MICHAEL J', 'LEFF DANIEL P', 'LAVORATO JOHN J', 'BERBERIAN DAVID', 'DETMERING TIM

I can see from this brief exploration that there are 146 exectives in the dataset, 18 identified POIs, and 22 features, for a total of 3088 observations. There are also a number of missing values. I'll investigate those in the next section.

### EDA and Outlier Removal

In [5]:
#change dataset to pandas dataframe
df = pandas.DataFrame.from_records(list(data_dict.values()))
employees = pandas.Series(list(data_dict.keys()))

#count number of NA values
df.replace(to_replace='NaN', value=numpy.nan, inplace=True)
print "Number of NaN values for each feature:"
print df.isnull().sum()
print "Shape of the dataframe: ", df.shape

Number of NaN values for each feature:
bonus                         64
deferral_payments            107
deferred_income               97
director_fees                129
email_address                 35
exercised_stock_options       44
expenses                      51
from_messages                 60
from_poi_to_this_person       60
from_this_person_to_poi       60
loan_advances                142
long_term_incentive           80
other                         53
poi                            0
restricted_stock              36
restricted_stock_deferred    128
salary                        51
shared_receipt_with_poi       60
to_messages                   60
total_payments                21
total_stock_value             20
dtype: int64
Shape of the dataframe:  (146, 21)


There are quite a lot of NaN values for some of the features. Particularly loan advances, director fees, restricted stock deferred, and deferral payments. However, when I look at the data schema provided (enron61702insiderpay.pdf), I see that these "missing values" are actually 0, so I will convert them to 0.

In [6]:
#replace missing values with 0
df.replace(to_replace=numpy.nan, value=0, inplace=True)
df.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,146.000000,146.000000,146.000000,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,146.000000,146.000000,1.460000e+02,1.460000e+02
mean,1.333474e+06,4.387965e+05,-3.827622e+05,1.942249e+04,4.182736e+06,7.074827e+04,358.602740,38.226027,24.287671,1.149658e+06,6.646839e+05,5.854318e+05,1.749257e+06,2.051637e+04,3.658114e+05,692.986301,1221.589041,4.350622e+06,5.846018e+06
std,8.094029e+06,2.741325e+06,2.378250e+06,1.190543e+05,2.607040e+07,4.327163e+05,1441.259868,73.901124,79.278206,9.649342e+06,4.046072e+06,3.682345e+06,1.089995e+07,1.439661e+06,2.203575e+06,1072.969492,2226.770637,2.693448e+07,3.624681e+07
min,0.000000e+00,-1.025000e+05,-2.799289e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.604490e+06,-7.576788e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,-4.409300e+04
25%,0.000000e+00,0.000000e+00,-3.792600e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.115000e+03,0.000000e+00,0.000000e+00,0.000000,0.000000,9.394475e+04,2.288695e+05
50%,3.000000e+05,0.000000e+00,0.000000e+00,0.000000e+00,6.082935e+05,2.018200e+04,16.500000,2.500000,0.000000,0.000000e+00,0.000000e+00,9.595000e+02,3.605280e+05,0.000000e+00,2.105960e+05,102.500000,289.000000,9.413595e+05,9.659550e+05
75%,8.000000e+05,9.684500e+03,0.000000e+00,0.000000e+00,1.714221e+06,5.374075e+04,51.250000,40.750000,13.750000,0.000000e+00,3.750648e+05,1.506065e+05,8.145280e+05,0.000000e+00,2.708505e+05,893.500000,1585.750000,1.968287e+06,2.319991e+06
max,9.734362e+07,3.208340e+07,0.000000e+00,1.398517e+06,3.117640e+08,5.235198e+06,14368.000000,528.000000,609.000000,8.392500e+07,4.852193e+07,4.266759e+07,1.303223e+08,1.545629e+07,2.670423e+07,5521.000000,15149.000000,3.098866e+08,4.345095e+08


In [7]:
#pairplot to visualize feature distributions
def splom_viz(df, labels=None):
    ax = sns.pairplot(df, hue="poi", diag_kind='kde', size=2, vars=['poi','salary', 'total_payments', 'bonus', 
                 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 
                 'restricted_stock', 'to_messages', 'from_poi_to_this_person', 'from_messages',
                 'from_this_person_to_poi', 'shared_receipt_with_poi'])
    plt.show()

#splom_viz(df)

Looking at this data, there definitely appears to be outliers. If I look at the data dictionary keys again, I see that there are two that are not names: Total and travel agency in the park. I'll remove these then look again at the pairplot.

#### Outlier Removal

In [8]:
#outlier removal
df= df.drop(df.index[[data_dict.keys().index("TOTAL"), data_dict.keys().index("THE TRAVEL AGENCY IN THE PARK")]])
df.describe()

#pairplot to visualize distributions and correllations
splom_viz(df)

After these outliers are removed, I can see on the pairplot that the most of the remainder of the outliers are classified as POI, so these "outliers" are in fact real data. The exception to this are the features 'from_poi_to_this_person', 'from_this_person_to_poi', and 'from_messages'. Looking at the statistics above, we see that the max 'from_messages' is 14368, which is one order of magnitude higher than the 75%. The same is true for the outliers in the other two categories.  Who are these people?

In [9]:
#identify keys for potential outliers
for key, value in data_dict.items():
    if value['from_poi_to_this_person'] != 'NaN' and value['from_poi_to_this_person'] > 500: 
        print "Max from_poi_to_this_person: ", key

for key, value in data_dict.items():
    if value['from_this_person_to_poi'] != 'NaN' and value['from_this_person_to_poi'] > 500: 
        print "Max from_this_person_to_poi: ", key
        
for key, value in data_dict.items():
    if value['from_messages'] != 'NaN' and value['from_messages'] > 14000: 
        print "Max from_messages: ", key

Max from_poi_to_this_person:  LAVORATO JOHN J
Max from_this_person_to_poi:  DELAINEY DAVID W
Max from_messages:  KAMINSKI WINCENTY J


Since these keys are all different people, I will keep the email data and assume it is real. Finally, looking at the pairplots and statistics for each feature, there are negative values for deferred_income, defferal_payments, restricted_stock, and restricted_stock deferred. Are these outliers, real data, or errors in the dataset?

#### Checking financial features

One final check we can do is make sure there aren't any mistakes in the financial data. In the data schema, we can see that two features: total_payments and total_stock_value are linear combinations of the other financial features. If the negative values observed above are real, then total_payments and total_stock_value should equal the sum of the values these features. If there is an error in the dataset, then these values will not be equal.

In [15]:
#Checking total pay
df_totalPay = pandas.DataFrame()
df_totalPay['name'] = data_dict.keys()
df_totalPay['total'] = (
    df['bonus'] + 
    df['director_fees'] +
    df['deferral_payments'] +
    df['deferred_income'] +
    df['loan_advances'] +
    df['long_term_incentive'] +
    df['expenses'] +
    df['other'] +
    df['salary']
)
df_totalPay['total_payments'] = df['total_payments']
df_totalPay['equals?'] = (df_totalPay['total'] == df_totalPay['total_payments'])
df_totalPay['poi'] = df['poi']
print numpy.sum(df_totalPay['equals?']), " out of ", len(df_totalPay)
print "Summed Totals Different than Total Payments"
print df_totalPay[df_totalPay['equals?'] == False]

142  out of  146
Summed Totals Different than Total Payments
                              name     total  total_payments equals?    poi
24                   BELFER ROBERT  -99215.0        102500.0   False  False
101  THE TRAVEL AGENCY IN THE PARK       NaN             NaN   False    NaN
104                          TOTAL       NaN             NaN   False    NaN
118               BHATNAGAR SANJAY  275728.0      15456290.0   False  False


In [16]:
#Checking total stock value
df_totalStock = pandas.DataFrame()
df_totalStock['name'] = data_dict.keys()
df_totalStock['total'] = (
    df['restricted_stock'] +
    df['exercised_stock_options'] +
    df['restricted_stock_deferred']
)
df_totalStock['total_stock_value'] = df['total_stock_value']
df_totalStock['equals?'] = (df_totalStock['total'] ==df_totalStock['total_stock_value'])
df_totalStock['poi'] = df['poi']
print numpy.sum(df_totalPay['equals?']), " out of ", len(df_totalPay)
print "Summed Totals Different from Total Stock Value"
print df_totalStock[df_totalStock['equals?'] == False]

142  out of  146
Summed Totals Different from Total Stock Value
                              name       total  total_stock_value equals?  \
24                   BELFER ROBERT     47378.0           -44093.0   False   
101  THE TRAVEL AGENCY IN THE PARK         NaN                NaN   False   
104                          TOTAL         NaN                NaN   False   
118               BHATNAGAR SANJAY  15456290.0                0.0   False   

       poi  
24   False  
101    NaN  
104    NaN  
118  False  


In [17]:
#check 
print data_dict["BELFER ROBERT"]
print"/n"
print data_dict["BHATNAGAR SANJAY"]

{'salary': 'NaN', 'to_messages': 'NaN', 'deferral_payments': -102500, 'total_payments': 102500, 'exercised_stock_options': 3285, 'bonus': 'NaN', 'restricted_stock': 'NaN', 'shared_receipt_with_poi': 'NaN', 'restricted_stock_deferred': 44093, 'total_stock_value': -44093, 'expenses': 'NaN', 'loan_advances': 'NaN', 'from_messages': 'NaN', 'other': 'NaN', 'from_this_person_to_poi': 'NaN', 'poi': False, 'director_fees': 3285, 'deferred_income': 'NaN', 'long_term_incentive': 'NaN', 'email_address': 'NaN', 'from_poi_to_this_person': 'NaN'}
/n
{'salary': 'NaN', 'to_messages': 523, 'deferral_payments': 'NaN', 'total_payments': 15456290, 'exercised_stock_options': 2604490, 'bonus': 'NaN', 'restricted_stock': -2604490, 'shared_receipt_with_poi': 463, 'restricted_stock_deferred': 15456290, 'total_stock_value': 'NaN', 'expenses': 'NaN', 'loan_advances': 'NaN', 'from_messages': 29, 'other': 137864, 'from_this_person_to_poi': 1, 'poi': False, 'director_fees': 137864, 'deferred_income': 'NaN', 'long_t

It appears that the values for these two employees are incorrect when I compare the data schema to the data dictionary. I will drop these two individuals from the dataframe.

In [19]:
#remove two outliers
df= df.drop(df.index[[data_dict.keys().index("BELFER ROBERT"), data_dict.keys().index("BHATNAGAR SANJAY")]])
df.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,1.400000e+02,1.400000e+02,1.400000e+02,140.000000,1.400000e+02,140.000000,140.000000,140.000000,140.000000,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,140.000000,140.000000,1.400000e+02,1.400000e+02
mean,6.745973e+05,2.291671e+05,-1.992171e+05,10242.007143,2.116031e+06,34692.071429,353.607143,39.114286,24.721429,5.994643e+05,3.395550e+05,3.031473e+05,8.860933e+05,7.520061e+04,1.851630e+05,707.342857,1227.507143,2.285244e+06,2.966923e+06
std,1.244090e+06,7.636494e+05,6.137650e+05,31707.700961,4.857146e+06,44067.333519,1457.460646,75.010415,80.689481,6.890811e+06,6.959270e+05,1.146372e+06,2.042272e+06,1.320536e+06,1.988771e+05,1085.716922,2259.344549,8.970016e+06,6.266642e+06
min,0.000000e+00,0.000000e+00,-3.504386e+06,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.604490e+06,-1.787380e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,-3.907200e+04,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.434500e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,8.747150e+04,2.443265e+05
50%,2.750000e+05,0.000000e+00,0.000000e+00,0.000000,6.082935e+05,20182.000000,16.500000,4.000000,0.000000,0.000000e+00,0.000000e+00,9.190000e+02,3.619780e+05,0.000000e+00,2.083105e+05,114.000000,289.000000,9.413595e+05,9.659550e+05
75%,8.000000e+05,1.184075e+04,0.000000e+00,0.000000,1.735597e+06,52471.750000,49.750000,41.250000,14.000000,0.000000e+00,3.745862e+05,1.505075e+05,8.571030e+05,0.000000e+00,2.696675e+05,933.750000,1543.250000,1.945668e+06,2.372703e+06
max,8.000000e+06,6.426990e+06,0.000000e+00,137864.000000,3.434838e+07,228763.000000,14368.000000,528.000000,609.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,1.545629e+07,1.111258e+06,5521.000000,15149.000000,1.035598e+08,4.911008e+07


In [ ]:
### scaling: use mixmax scaler to normalize all features between 0 and 1. This will weight POI greater than all other features. That's what we want.

### Feature Selection

Before I look at selecting the features for my classifier, I'll first create a new feature.

In [ ]:
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)